<a href="https://colab.research.google.com/github/tejkrish22/learn_git/blob/master/Copy_of_rec_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

def load_data(filename):
    df = pd.read_csv(filename, header=None)
    data = df.iloc[:, :2]
    return data.values
filename = '/content/drive/MyDrive/train/myArray.csv'
data = load_data(filename)

In [61]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
seq_len = 10
padded_data = tf.keras.preprocessing.sequence.pad_sequences([scaled_data], maxlen=seq_len, padding='pre', dtype='float32')
n_features = 2
hidden_dim=128

In [ ]:
# prepare recurrent auto encoder and train it
in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
gru1 = tf.keras.layers.GRU(hidden_dim, name='latent_layer')(in1)
rv1 = tf.keras.layers.RepeatVector(seq_len)(gru1)
gru2 = tf.keras.layers.GRU(hidden_dim, return_sequences=True)(rv1)
tdd1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(gru2) 
model = tf.keras.Model(inputs=in1, outputs=tdd1, name='RAE')
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='mse')
history = model.fit(padded_data, padded_data, epochs=50, batch_size=64)
# Extract encoder model
encoder = tf.keras.Model(inputs=in1, outputs=gru1)

In [79]:
temp = data[0:10]
temp = np.expand_dims(temp,axis=0)
temp.shape
X_encoded = encoder.predict(temp)
X_encoded = np.append(X_encoded,[[1,2,3]],axis=1)
print(X_encoded.shape)

1/1 [==============================] - 0s 30ms/step
(1, 131)


In [57]:
for i in range(len(data)):
  temp = scaled_data[i:i+10]
  temp = np.expand_dims(temp,axis=0)     # x . y . t . w . d . s . l 
  X_encoded = encoder.predict(temp)
  X_encoded = np.append(X_encoded,[data[i][3],data[i][4],data[i][6]],axis=1)
  

10646823

In [ ]:


# Encode data using encoder model
X_encoded = encoder.predict(padded_data)

encoder.summary()

# embedding (128), writer id, doc id, text line id
# row denotes the sub stroke no, col denotes the sub no.

1/1 [==============================] - 1s 850ms/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 2)]           0         
                                                                 
 latent_layer (GRU)          (None, 128)               50688     
                                                                 
Total params: 50,688
Trainable params: 50,688
Non-trainable params: 0
_________________________________________________________________


In [ ]:
padded_data.shape

(1, 10, 2)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Reshape
from tensorflow.keras.callbacks import EarlyStopping
# Define LSTM model
input_shape = (seq_len, hidden_dim)
input_layer = Input(shape=input_shape)
lstm_layer = LSTM(64, activation='relu', return_sequences=True)(input_layer)
output_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(lstm_layer)
lstm_model = Model(inputs=input_layer, outputs=output_layer)
lstm_model.compile(optimizer='adam', loss='mse')

# Train LSTM model
history = lstm_model.fit(X_encoded, padded_data, epochs=50, batch_size=32,
                         callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

